In [ ]:
import jax
import jax.numpy as jnp
from ..envs.realistic.RealisticPendulum import RealisticPendulum

trajectory_length = 100
num_samples = 18000

In [ ]:
def generate_inputs(rng,trajectory_length):
    return jax.random.uniform(rng, (trajectory_length, 1), minval=-3, maxval=3)

In [ ]:
Pendulum = RealisticPendulum()
observation_t = None
observation_tp = None
action_t = None

for i in range(num_samples):
    #generate RNG
    rnjesus = jax.random.key(i)
    inputs = generate_inputs(rnjesus, trajectory_length)
    state = Pendulum.reset(rng=rnjesus)
    observations = jnp.zeros((trajectory_length, 4))
    print(i)
    for t in range(trajectory_length):
        action = inputs[t]
        state = Pendulum.step(state, action)
        observations = observations.at[t].set(jnp.concatenate([state.pipeline_state.q, state.pipeline_state.qd]))
        #print(t)
   
    if observation_t == None:
        observation_t = observations[:-1]
        observation_tp = observations[1:]
        action_t = inputs[:-1]
        print(observation_t.shape, observation_tp.shape, action_t.shape)
    else:
        observation_t = jnp.concatenate([observation_t, observations[:-1]])
        observation_tp = jnp.concatenate([observation_tp, observations[1:]])
        action_t = jnp.concatenate([action_t, inputs[:-1]])

jnp.savez("Data/data.npz", observation_t=observation_t, observation_tp=observation_tp, action_t=action_t)
